Os dados coletados são do Sistema de Informação Ambulatorial (SIA) do DATASUS [basedosdados](https://basedosdados.org/dataset/22d1f0d6-9bbc-4653-a841-7734867d2319?table=f68affc8-b62a-481e-a61a-abe900f9df16). Os dados foram filtrados por estado e região para coleta apenas dos referentes à Curitba e Região Metropolitana. 

**Definição do problema:** Analise do volume de atendimentos psicossociais (procedimentos realizados) registrados na rede ambulatorial (SIA) de Curitiba e Região Metropolitana nos últimos 10 anos. Identificar os grupos que mais utilizam os serviços (faixa etária, raça e região)

## 1. Configuração do ambiente e carregamento de dados

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [2]:
# Carregamento da tabela principal
caminho = '../data/raw/dados_brutos.csv'
df_psico = pd.read_csv(caminho)

In [3]:
# Carregamento de dicionários
dic_cid = pd.read_csv('../data/dicionarios/br_bd_diretorios_brasil_cid_10.csv')
dic_municipio = pd.read_csv('../data/dicionarios/br_bd_diretorios_brasil_municipio.csv')
dic_geral = pd.read_csv('../data/dicionarios/br_ms_sia_dicionario.csv')

## 2. Entendimento e limpeza de dados

### 2.1. Limpeza inicial dos dados

In [5]:
df_psico.columns

Index(['ano', 'mes', 'sigla_uf', 'id_municipio', 'id_estabelecimento_cnes',
       'id_estabelecimento_cnes_familia', 'id_procedimento_ambulatorial',
       'id_servico_especializado', 'id_classificacao_servico',
       'data_inicio_atendimento', 'data_termino_atendimento',
       'permanencia_atendimento', 'motivo_saida_permanencia',
       'data_motivo_saida_permanencia', 'ano_processamento',
       'mes_processamento', 'ano_atendimento', 'mes_atendimento',
       'data_nascimento_paciente', 'id_municipio_residencia_paciente',
       'origem_paciente', 'nacionalidade_paciente', 'tipo_idade',
       'idade_paciente', 'sexo_paciente', 'raca_cor_paciente',
       'etnia_paciente', 'carater_atendimento', 'cid_principal_categoria',
       'cid_principal_subcategoria', 'cid_causas_associadas_categoria',
       'cid_causas_associadas_subcategoria', 'tipo_droga', 'destino_paciente',
       'local_realizacao_atendimento', 'indicador_situacao_rua',
       'indicador_estrategia_familia', 'quant

In [6]:
df_psico.shape

(3717639, 41)

In [7]:
df_psico['ano_atendimento'].unique()

array([2012, 2014, 2015, 2017, 2018, 2019, 2021, 2013, 2016, 2022, 2023,
       2020, 2024, 2025])

Remoção de dados fora da análise temporal:

In [8]:
remover = [2012, 2013, 2025]
df_psico = df_psico[~df_psico['ano_atendimento'].isin(remover)]

Identificação e tratamento de nulos:

In [9]:
nulos = (df_psico.isnull().sum() / len(df_psico)) * 100 
colunas_nulas = nulos[nulos > 0].sort_values(ascending=False)

print("Percentual de valores nulos por coluna:")
print(colunas_nulas)

Percentual de valores nulos por coluna:
indicador_situacao_rua                100.000000
indicador_estrategia_familia          100.000000
data_motivo_saida_permanencia          98.490448
cid_causas_associadas_categoria        97.649287
cid_causas_associadas_subcategoria     93.884226
data_inicio_atendimento                79.061244
data_termino_atendimento               79.061244
cid_principal_categoria                63.873240
cid_principal_subcategoria             36.126760
dtype: float64


In [10]:
# Tratamento de nulos das colunas referentes ao CID
df_psico['cid_categoria_geral'] = df_psico['cid_principal_subcategoria'].fillna(df_psico['cid_principal_categoria'])
df_psico['cid_categoria_geral'].isnull().mean().round(2)

np.float64(0.0)

In [11]:
df_psico = df_psico.drop(columns=['cid_principal_categoria', 'cid_principal_subcategoria'])

Remoção de colunas que não são incluídas na análise e colunas nulas:

In [12]:
remover = ['indicador_situacao_rua', 'indicador_estrategia_familia', 'data_motivo_saida_permanencia', 'cid_causas_associadas_categoria', 
           'cid_causas_associadas_subcategoria', 'local_realizacao_atendimento', 'sigla_uf', 'nacionalidade_paciente', 'origem_paciente', 
           'tipo_idade', 'ano_processamento', 'id_estabelecimento_cnes_familia', 'id_classificacao_servico', 'mes_processamento', 'permanencia_atendimento', 
           'motivo_saida_permanencia', 'etnia_paciente', 'carater_atendimento', 'tipo_droga', 'destino_paciente', 'ano', 'mes', 'id_servico_especializado', 
           'data_termino_atendimento'
        ]
df_psico = df_psico.drop(columns=remover)

In [13]:
df_psico = df_psico[df_psico['quantidade_aprovada_procedimento'] > 0].copy()

In [14]:
df_psico = df_psico[df_psico['quantidade_pacientes'] == 1].copy()

In [15]:
df_psico = df_psico.drop(columns=['quantidade_pacientes'])

In [16]:
df_psico.shape

(652360, 15)

In [17]:
df_psico.head()

,id_municipio,id_estabelecimento_cnes,id_procedimento_ambulatorial,data_inicio_atendimento,ano_atendimento,mes_atendimento,data_nascimento_paciente,id_municipio_residencia_paciente,idade_paciente,sexo_paciente,raca_cor_paciente,quantidade_produzida_procedimento,quantidade_aprovada_procedimento,quantidade_atendimentos,cid_categoria_geral
5,4125506,3709256,301080208,2014-02-07,2014,2,1986-09-09,4125506,27,M,99,1,1,1,F102
6,4113205,3904334,301080240,2014-02-01,2014,2,1967-01-27,4113205,46,M,3,1,1,1,F102
7,4119509,5075483,301080194,2014-02-27,2014,2,1980-04-09,4119509,33,F,3,1,1,1,F430
8,4119509,5075483,301080208,2014-02-01,2014,2,1973-07-14,4119509,39,F,3,1,1,1,F45
9,4119509,5075483,301080194,2014-02-14,2014,2,1954-10-23,4119509,59,F,3,1,1,1,F32


### 2.1. Junção com dicionários

In [18]:
# Nome dos municípios de atendimento e residência do paciente
dic_nomes = dic_municipio.set_index('id_municipio')['nome']
df_psico['municipio_atendimento'] = df_psico['id_municipio'].map(dic_nomes)
df_psico['municipio_residencia'] = df_psico['id_municipio_residencia_paciente'].map(dic_nomes)

In [19]:
df_psico = df_psico.drop(columns=['id_municipio', 'id_municipio_residencia_paciente'])

In [20]:
colunas_chave = ['nome_coluna', 'chave']
dic_geral = dic_geral.drop_duplicates(subset=colunas_chave, keep='first')

In [21]:
raca_cor = dic_geral[dic_geral['nome_coluna'] == 'raca_cor_paciente']
mapa = raca_cor.set_index('chave')['valor']
df_psico['raca_cor_paciente'] = df_psico['raca_cor_paciente'].astype(str).map(mapa)

In [22]:
# Remoção de pacientes não-residentes da região de análise
df_municipios_validos = ['Curitiba', 'Adrianópolis', 'Agudos do Sul', 'Almirante Tamandaré', 'Araucária', 'Balsa Nova','Bocaiúva do Sul', 
                         'Campina Grande do Sul', 'Campo do Tenente','Campo Largo', 'Campo Magro', 'Cerro Azul', 'Colombo', 'Contenda', 
                         'Doutor Ulysses', 'Fazenda Rio Grande', 'Itaperuçu', 'Lapa', 'Mandirituba', 'Piên', 'Pinhais', 'Piraquara', 'Quatro Barras', 
                         'Rio Branco do Sul', 'Rio Negro', 'São José dos Pinhais', 'Quitandinha', 'Tijucas do Sul', 'Tunas do Paraná']

df_psico = df_psico[df_psico['municipio_residencia'].isin(df_municipios_validos)]

In [23]:
df_psico['raca_cor_paciente'].unique()

array(['sem informacao', 'parda', 'branca', 'preta', 'amarela',
       'indigena'], dtype=object)

In [24]:
df_psico['raca_cor_paciente'] = df_psico['raca_cor_paciente'].replace('sem informacao', 'não declarado')

### 2.3. Estatística e tratamento dos dados

#### Análise de inconsistências

In [25]:
df_psico.info()

<class 'pandas.core.frame.DataFrame'>
Index: 651745 entries, 5 to 3717638
Data columns (total 15 columns):
 #   Column                             Non-Null Count   Dtype 
---  ------                             --------------   ----- 
 0   id_estabelecimento_cnes            651745 non-null  int64 
 1   id_procedimento_ambulatorial       651745 non-null  int64 
 2   data_inicio_atendimento            651745 non-null  object
 3   ano_atendimento                    651745 non-null  int64 
 4   mes_atendimento                    651745 non-null  int64 
 5   data_nascimento_paciente           651745 non-null  object
 6   idade_paciente                     651745 non-null  int64 
 7   sexo_paciente                      651745 non-null  object
 8   raca_cor_paciente                  651745 non-null  object
 9   quantidade_produzida_procedimento  651745 non-null  int64 
 10  quantidade_aprovada_procedimento   651745 non-null  int64 
 11  quantidade_atendimentos            651745 non-null  int6

In [26]:
df_psico = df_psico.drop(columns=['quantidade_produzida_procedimento'])

In [27]:
# Verificação de consistência
meses = df_psico['mes_atendimento'].unique()
meses.sort()

sexo = df_psico['sexo_paciente'].unique()
nascimento = df_psico['data_nascimento_paciente'].isnull().sum()

print(f'Meses: {meses}')
print(f'Sexo: {sexo}')
print(f'Data de nascimento nulas: {nascimento}')

Meses: [ 1  2  3  4  5  6  7  8  9 10 11 12]
Sexo: ['M' 'F']
Data de nascimento nulas: 0


In [28]:
pacientes = df_psico[df_psico['idade_paciente'] < 2]
columns = ['ano_atendimento', 'data_nascimento_paciente', 'idade_paciente']

pacientes[columns].head()

,ano_atendimento,data_nascimento_paciente,idade_paciente
1052,2014,2013-11-29,0
1123,2014,1962-01-13,0
2253,2014,2014-07-13,1
2260,2014,2014-06-05,0
3105,2015,2007-02-01,0


Percebe-se uma inconsistência nas idades dos pacientes

In [29]:
data_estimada_atendimento = pd.to_datetime(df_psico['ano_atendimento'].astype(str) + '-' + \
                                           df_psico['mes_atendimento'].astype(str) + '-01')
                         
data_atendimento_completa = pd.to_datetime(df_psico['data_inicio_atendimento'])
data_referencia = data_atendimento_completa.fillna(data_estimada_atendimento)

data_nascimento = pd.to_datetime(df_psico['data_nascimento_paciente'])

idade = data_referencia.dt.year - data_nascimento.dt.year

ajuste = (data_referencia.dt.month < data_nascimento.dt.month) | \
                               ((data_referencia.dt.month == data_nascimento.dt.month) & \
                                (data_referencia.dt.day < data_nascimento.dt.day))

df_psico['idade_paciente'] = idade - ajuste

In [30]:
pacientes = df_psico[df_psico['idade_paciente'] < 2]
columns = ['ano_atendimento', 'data_nascimento_paciente', 'idade_paciente']

pacientes[columns].head()

,ano_atendimento,data_nascimento_paciente,idade_paciente
966,2014,2014-01-17,0
1052,2014,2013-11-29,0
2253,2014,2014-07-13,0
2260,2014,2014-06-05,0
3115,2015,2014-10-28,0


Foi calculado a idade exata de 20% dos dados (dados início atendimento não-nulos) e a idade aproximada do restante considerando o primeiro dia do mês como o dia de atendimento

In [31]:
print(df_psico['idade_paciente'].min())
print(df_psico['idade_paciente'].max())

0
114


In [33]:
# Faixa etaria:
bins_ciclos = [0, 12, 18, 29, 59, 150]
labels_ciclos = ['Criança (0-12)', 'Adolescente (13-18)', 'Jovem Adulto (19-29)', 'Adulto (30-59)', 'Idoso (60+)']

df_psico['faixa_etaria'] = pd.cut(
    df_psico['idade_paciente'],
    bins=bins_ciclos,
    labels=labels_ciclos,
    right=True,        
    include_lowest=True
)
print(df_psico['faixa_etaria'].value_counts().sort_index())

faixa_etaria
Criança (0-12)           44432
Adolescente (13-18)      80579
Jovem Adulto (19-29)    106116
Adulto (30-59)          372815
Idoso (60+)              47803
Name: count, dtype: int64


In [34]:
mapa_faixas = dict(enumerate(df_psico['faixa_etaria'].cat.categories))
df_psico['id_faixa_etaria'] = df_psico['faixa_etaria'].cat.codes

#### Tratamento de outliners

In [35]:
df_psico.shape

(651745, 16)

In [36]:
df_psico.describe()

,id_estabelecimento_cnes,id_procedimento_ambulatorial,ano_atendimento,mes_atendimento,idade_paciente,quantidade_aprovada_procedimento,quantidade_atendimentos,id_faixa_etaria
count,6.517450e+05,6.517450e+05,651745.000000,651745.000000,651745.000000,651745.000000,651745.0,651745.000000
mean,5.250449e+06,3.010802e+08,2019.518738,6.640609,36.492426,1.361511,1.0,2.458735
std,1.653894e+06,4.121732e+01,3.183101,3.424466,16.382116,2.749944,0.0,1.025332
min,1.529600e+04,3.010800e+08,2014.000000,1.000000,0.000000,1.000000,1.0,0.000000
25%,3.739112e+06,3.010802e+08,2017.000000,4.000000,23.000000,1.000000,1.0,2.000000
50%,5.319226e+06,3.010802e+08,2019.000000,7.000000,37.000000,1.000000,1.0,3.000000
75%,6.569749e+06,3.010802e+08,2022.000000,10.000000,49.000000,1.000000,1.0,3.000000
max,9.695915e+06,3.010804e+08,2024.000000,12.000000,114.000000,496.000000,1.0,4.000000


In [38]:
media = df_psico['quantidade_aprovada_procedimento'].mean()
desvio = df_psico['quantidade_aprovada_procedimento'].std()

cv = (media/desvio) * 100
print(cv.round(2))

49.51


Já percebe-se valores extremos nas colunas quantidade aprovada de procedimento

In [39]:
percentil = df_psico['quantidade_aprovada_procedimento'].quantile(0.999, interpolation='nearest')
print(percentil)

36


In [43]:
df_psico = df_psico[df_psico['quantidade_aprovada_procedimento'] <= percentil].copy()

In [44]:
df_psico.shape

(651118, 16)

#### Consolidação dos dados e exportação

In [45]:
colunas = [
    'id_estabelecimento_cnes', 'id_procedimento_ambulatorial', 'ano_atendimento', 'mes_atendimento',
    'sexo_paciente','raca_cor_paciente', 'cid_categoria_geral', 'municipio_atendimento',
    'municipio_residencia', 'id_faixa_etaria'
]
metricas= {
    'quantidade_atendimentos': 'sum',
    'quantidade_aprovada_procedimento': 'sum' 
}
df_analise = df_psico.groupby(colunas, as_index=False).agg(metricas)

In [46]:
df_analise['faixa_etaria'] = df_analise['id_faixa_etaria'].map(mapa_faixas)

In [47]:
df_analise = df_analise.drop(columns=['id_faixa_etaria'])

In [48]:
df_analise.head()

,id_estabelecimento_cnes,id_procedimento_ambulatorial,ano_atendimento,mes_atendimento,sexo_paciente,raca_cor_paciente,cid_categoria_geral,municipio_atendimento,municipio_residencia,quantidade_atendimentos,quantidade_aprovada_procedimento,faixa_etaria
0,15296,301080208,2014,1,F,branca,F701,Curitiba,Fazenda Rio Grande,1,1,Adolescente (13-18)
1,15296,301080208,2014,1,F,branca,F911,Curitiba,Campo Magro,1,1,Adolescente (13-18)
2,15296,301080208,2014,1,F,parda,F929,Curitiba,Piraquara,1,1,Adolescente (13-18)
3,15296,301080208,2014,1,M,branca,F711,Curitiba,Fazenda Rio Grande,1,1,Adolescente (13-18)
4,15296,301080208,2014,1,M,branca,F840,Curitiba,Colombo,1,1,Adolescente (13-18)


In [49]:
df_analise.describe()

,id_estabelecimento_cnes,id_procedimento_ambulatorial,ano_atendimento,mes_atendimento,quantidade_atendimentos,quantidade_aprovada_procedimento
count,3.424610e+05,3.424610e+05,342461.000000,342461.000000,342461.000000,342461.000000
mean,5.233916e+06,3.010802e+08,2019.596973,6.651759,1.901291,2.471090
std,1.735401e+06,4.589136e+01,3.224881,3.431287,3.223049,6.537291
min,1.529600e+04,3.010800e+08,2014.000000,1.000000,1.000000,1.000000
25%,3.903893e+06,3.010802e+08,2017.000000,4.000000,1.000000,1.000000
50%,5.319226e+06,3.010802e+08,2019.000000,7.000000,1.000000,1.000000
75%,6.374352e+06,3.010802e+08,2023.000000,10.000000,2.000000,2.000000
max,9.695915e+06,3.010804e+08,2024.000000,12.000000,119.000000,570.000000
